In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Automatic Differentation and Gradients

x = tf.Variable(3.0)

with tf.GradientTape() as tape:
    y = x**2

    dy_dx = tape.gradient(y,x)
    dy_dx.numpy()

print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)


In [6]:
w = tf.Variable(tf.random.normal((3,2), name='w'))
b = tf.Variable(tf.zeros(2, dtype=tf.float32), name='b')
x = [[1.0, 2.0, 3.0]]

print(w.numpy())
print(b.numpy())

with tf.GradientTape(persistent=True) as tape:
    y = x @ w + b
    loss = tf.reduce_mean(y**2)

[dl_dw, dl_db] = tape.gradient(loss, [w,b])

print(w.shape)
print(w.numpy())
print(dl_dw.shape)
print(dl_dw.numpy())


[[ 0.42177057  1.0352646 ]
 [-2.2723117   0.27003923]
 [ 0.6620311  -1.3860588 ]]
[0. 0.]
(3, 2)
[[ 0.42177057  1.0352646 ]
 [-2.2723117   0.27003923]
 [ 0.6620311  -1.3860588 ]]
(3, 2)
[[-2.1367595 -2.5828335]
 [-4.273519  -5.165667 ]
 [-6.4102783 -7.748501 ]]
